***GENERATED CODE FOR loanstatus PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'address_state', 'transformation_label': 'String Indexer'}], 'feature': 'address_state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WY', 'missing': '0', 'distinct': '42'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'address_state'}, {'feature_label': 'address_state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('address_state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'application_type', 'transformation_label': 'String Indexer'}], 'feature': 'application_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'INDIVIDUAL', 'max': 'INDIVIDUAL', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'application_type'}, {'feature_label': 'application_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('application_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'emp_length', 'transformation_label': 'String Indexer'}], 'feature': 'emp_length', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 year', 'max': '< 1 year', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'emp_length'}, {'feature_label': 'emp_length', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('emp_length')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'emp_title', 'transformation_label': 'String Indexer'}], 'feature': 'emp_title', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': '1st Class Sleep Diagnostics, Inc.', 'max': 'wny urology associates', 'missing': '10', 'distinct': '475'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'emp_title'}, {'feature_label': 'emp_title', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('emp_title')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'grade', 'transformation_label': 'String Indexer'}], 'feature': 'grade', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'grade'}, {'feature_label': 'grade', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('grade')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'home_ownership', 'transformation_label': 'String Indexer'}], 'feature': 'home_ownership', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'MORTGAGE', 'max': 'RENT', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'home_ownership'}, {'feature_label': 'home_ownership', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('home_ownership')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'issue_date', 'transformation_label': 'String Indexer'}], 'feature': 'issue_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10-05-2021', 'max': '11-12-2021', 'missing': '0', 'distinct': '61'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'issue_date'}, {'feature_label': 'issue_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('issue_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'last_credit_pull_date', 'transformation_label': 'String Indexer'}], 'feature': 'last_credit_pull_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '07-08-2021', 'max': '16-05-2021', 'missing': '0', 'distinct': '80'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'last_credit_pull_date'}, {'feature_label': 'last_credit_pull_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('last_credit_pull_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'last_payment_date', 'transformation_label': 'String Indexer'}], 'feature': 'last_payment_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10-07-2021', 'max': '16-05-2021', 'missing': '0', 'distinct': '66'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'last_payment_date'}, {'feature_label': 'last_payment_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('last_payment_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan_status', 'transformation_label': 'String Indexer'}], 'feature': 'loan_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Charged Off', 'max': 'Fully Paid', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan_status'}, {'feature_label': 'loan_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan_status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'next_payment_date', 'transformation_label': 'String Indexer'}], 'feature': 'next_payment_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '08-01-2022', 'max': '16-04-2021', 'missing': '0', 'distinct': '66'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'next_payment_date'}, {'feature_label': 'next_payment_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('next_payment_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'purpose', 'transformation_label': 'String Indexer'}], 'feature': 'purpose', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Debt consolidation', 'max': 'Debt consolidation', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'purpose'}, {'feature_label': 'purpose', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('purpose')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sub_grade', 'transformation_label': 'String Indexer'}], 'feature': 'sub_grade', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A1', 'max': 'E1', 'missing': '0', 'distinct': '31'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sub_grade'}, {'feature_label': 'sub_grade', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sub_grade')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'term', 'transformation_label': 'String Indexer'}], 'feature': 'term', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' 36 months', 'max': ' 60 months', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'term'}, {'feature_label': 'term', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('term')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'verification_status', 'transformation_label': 'String Indexer'}], 'feature': 'verification_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Source Verified', 'max': 'Source Verified', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'verification_status'}, {'feature_label': 'verification_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('verification_status')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'annual_income', 'threshold': 60244.996, 'transformation_label': 'Binarizer'}], 'feature': 'annual_income', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '71827.05', 'stddev': '52207.54', 'min': '8000.0', 'max': '720000.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'annual_income'}, {'feature_label': 'annual_income', 'threshold': 60244.996, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('annual_income')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run loanstatusHooks.ipynb
try:
	#sourcePreExecutionHook()

	financialloan = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/financial_loan.csv', 'filename': 'financial_loan.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run loanstatusHooks.ipynb
try:
	#transformationPreExecutionHook()

	loanstatusautofe = TransformationMain.run(financialloan,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "640713.5", "stddev": "221167.67", "min": "65426", "max": "1077430", "missing": "0"}, "updatedLabel": "id"}, {"transformationsData": [{"feature_label": "address_state", "transformation_label": "String Indexer"}], "feature": "address_state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AL", "max": "WY", "missing": "0", "distinct": "42"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "address_state"}, {"transformationsData": [{"feature_label": "application_type", "transformation_label": "String Indexer"}], "feature": "application_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "INDIVIDUAL", "max": "INDIVIDUAL", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "application_type"}, {"transformationsData": [{"feature_label": "emp_length", "transformation_label": "String Indexer"}], "feature": "emp_length", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 year", "max": "< 1 year", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "emp_length"}, {"transformationsData": [{"feature_label": "emp_title", "transformation_label": "String Indexer"}], "feature": "emp_title", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "1st Class Sleep Diagnostics, Inc.", "max": "wny urology associates", "missing": "10", "distinct": "475"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "emp_title"}, {"transformationsData": [{"feature_label": "grade", "transformation_label": "String Indexer"}], "feature": "grade", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "grade"}, {"transformationsData": [{"feature_label": "home_ownership", "transformation_label": "String Indexer"}], "feature": "home_ownership", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "MORTGAGE", "max": "RENT", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "home_ownership"}, {"transformationsData": [{"feature_label": "issue_date", "transformation_label": "String Indexer"}], "feature": "issue_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10-05-2021", "max": "11-12-2021", "missing": "0", "distinct": "61"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "issue_date"}, {"transformationsData": [{"feature_label": "last_credit_pull_date", "transformation_label": "String Indexer"}], "feature": "last_credit_pull_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "07-08-2021", "max": "16-05-2021", "missing": "0", "distinct": "80"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "last_credit_pull_date"}, {"transformationsData": [{"feature_label": "last_payment_date", "transformation_label": "String Indexer"}], "feature": "last_payment_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10-07-2021", "max": "16-05-2021", "missing": "0", "distinct": "66"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "last_payment_date"}, {"transformationsData": [{"feature_label": "loan_status", "transformation_label": "String Indexer"}], "feature": "loan_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Charged Off", "max": "Fully Paid", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan_status"}, {"transformationsData": [{"feature_label": "next_payment_date", "transformation_label": "String Indexer"}], "feature": "next_payment_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "08-01-2022", "max": "16-04-2021", "missing": "0", "distinct": "66"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "next_payment_date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "member_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "792543.24", "stddev": "285940.0", "min": "112227", "max": "1314167", "missing": "0"}, "updatedLabel": "member_id"}, {"transformationsData": [{"feature_label": "purpose", "transformation_label": "String Indexer"}], "feature": "purpose", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Debt consolidation", "max": "Debt consolidation", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "purpose"}, {"transformationsData": [{"feature_label": "sub_grade", "transformation_label": "String Indexer"}], "feature": "sub_grade", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A1", "max": "E1", "missing": "0", "distinct": "31"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sub_grade"}, {"transformationsData": [{"feature_label": "term", "transformation_label": "String Indexer"}], "feature": "term", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " 36 months", "max": " 60 months", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "term"}, {"transformationsData": [{"feature_label": "verification_status", "transformation_label": "String Indexer"}], "feature": "verification_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Source Verified", "max": "Source Verified", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "verification_status"}, {"transformationsData": [{"feature_label": "annual_income", "threshold": 60244.996, "transformation_label": "Binarizer"}], "feature": "annual_income", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "71827.05", "stddev": "52207.54", "min": "8000.0", "max": "720000.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "annual_income"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dti", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.11", "stddev": "0.07", "min": "0.0", "max": "0.2935", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dti"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "installment", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "191.48", "stddev": "117.04", "min": "30.88", "max": "817.63", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "installment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "int_rate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.14", "stddev": "0.03", "min": "0.0617", "max": "0.2352", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "int_rate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "loan_amount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6064.5", "stddev": "3586.73", "min": "1000", "max": "24000", "missing": "0"}, "updatedLabel": "loan_amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_acc", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "20.17", "stddev": "11.16", "min": "3", "max": "63", "missing": "0"}, "updatedLabel": "total_acc"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_payment", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "13556.02", "stddev": "9373.86", "min": "375", "max": "49708", "missing": "0"}, "updatedLabel": "total_payment"}]}))

	#transformationPostExecutionHook(loanstatusautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run loanstatusHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(loanstatusautofe, ["id", "member_id", "dti", "installment", "int_rate", "loan_amount", "total_acc", "total_payment", "address_state_stringindexer", "application_type_stringindexer", "emp_length_stringindexer", "emp_title_stringindexer", "grade_stringindexer", "home_ownership_stringindexer", "issue_date_stringindexer", "last_credit_pull_date_stringindexer", "last_payment_date_stringindexer", "next_payment_date_stringindexer", "purpose_stringindexer", "sub_grade_stringindexer", "term_stringindexer", "verification_status_stringindexer", "annual_income_binarizer"], "loan_status_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

